In [75]:
rsi_win = 18; rsi_buy = 46; rsi_sell = 52; num_atr_loss = 10

market = 'BTC-XVG'
tf = '5m'
robot_name = 'rsi_01'
from robots_garage import rsi_01

start_capital = 0.01
deposit_percent = 0.3

#Work01
#api_key = "c4371a7d59f64a59b5667c3daaba255e"
#api_secret = "5cc6d8e168914ecd91f6f2136ec0310f"

#Work 30
api_key = "mZrSEDmM7GJ7dCBT5oKcTa5pKdV8CXeEQoR4nlLGPTBWtn9GWqi0UFXLyFG8E6Ry"
api_secret = "bLtJGjMH0gRRtvDa7OXifGlgpFM2NrsbmOXBnt8tyUuigShZwm2MissRO2PeAKeT"

#Work 5
#api_key = "7761af64992b4b2596b4f0d921697d51"
#api_secret = "e9f4062734d64d698bb024287ab19720"


In [76]:
import pandas as pd
import ccxt

from data_provider import read_data_path

pd.set_option('precision', 9)
exchange = ccxt.binance({'apiKey': api_key,
                         'secret': api_secret})

In [80]:
exchange.fetchOrders(symbol)

[{'amount': 500.0,
  'cost': 0.0,
  'datetime': '2018-04-14T09:24:37.345Z',
  'fee': None,
  'filled': 0.0,
  'id': '27270366',
  'info': {'clientOrderId': 'ngacZTsmbDLZ2OyaaAEKxs',
   'executedQty': '0.00000000',
   'icebergQty': '0.00000000',
   'isWorking': True,
   'orderId': 27270366,
   'origQty': '500.00000000',
   'price': '0.00001000',
   'side': 'BUY',
   'status': 'CANCELED',
   'stopPrice': '0.00000000',
   'symbol': 'XVGBTC',
   'time': 1523697877345,
   'timeInForce': 'GTC',
   'type': 'LIMIT'},
  'price': 1e-05,
  'remaining': 500.0,
  'side': 'buy',
  'status': 'canceled',
  'symbol': 'XVG/BTC',
  'timestamp': 1523697877345,
  'type': 'limit'}]

In [78]:
print(order['id'])

exchange.cancel_order('27270366', symbol)

27270366


{'clientOrderId': 'S2qyTMtd1NicRBkDWmaWXx',
 'orderId': 27270366,
 'origClientOrderId': 'ngacZTsmbDLZ2OyaaAEKxs',
 'symbol': 'XVGBTC'}

In [45]:
symbol = 'XVG/BTC'
type = 'limit'  # or 'market'
side = 'buy'  # or 'buy'
amount = 500.0
price = 0.00001  # or None

# extra params and overrides if needed
params = {
    'test': True,  # test if it's valid, but don't actually place it
}

#order = exchange.create_order(symbol, type, side, amount, price)
order = exchange.create_limit_buy_order (symbol, amount, price)
print(order)

{'info': {'symbol': 'XVGBTC', 'orderId': 27270366, 'clientOrderId': 'ngacZTsmbDLZ2OyaaAEKxs', 'transactTime': 1523697877345, 'price': '0.00001000', 'origQty': '500.00000000', 'executedQty': '0.00000000', 'status': 'NEW', 'timeInForce': 'GTC', 'type': 'LIMIT', 'side': 'BUY'}, 'id': '27270366', 'timestamp': 1523697877345, 'datetime': '2018-04-14T09:24:37.345Z', 'symbol': 'XVG/BTC', 'type': 'limit', 'side': 'buy', 'price': 1e-05, 'amount': 500.0, 'cost': 0.0, 'filled': 0.0, 'remaining': 500.0, 'status': 'open', 'fee': None}


In [9]:
def response_fail(response):
    if response['result'] == None or response['result'] == [] :
        return True
    else:
        return False

#Происходит неявное накопление за счет расчета numShares
def get_price_orderbook(action, dp = dp11):
# API2.0 неправильно отрабатывает ордерную книгу, поэтому применяем 1.1    
    if action == 'Entry':
        order_type = 'sell'
    else:
        order_type = 'buy'
    orders = dp.get_orderbook(market_name)
    orders = json_normalize(orders['result'][order_type])[:10]
    price = orders['Rate'][3]
    return price

def get_price_ticker(action, dp = dp11):
# API2.0 неправильно отрабатывает ордерную книгу, поэтому применяем 1.1    
    if action == 'Entry':
        order_type = 'Ask'
    else:
        order_type = 'Bid'
    ticker = dp.get_ticker(market_name)
    price = json_normalize(ticker['result'])
    price = price[order_type][0]
    return price

def write_trades(df_trades, trade):
    print('{d[0]} {d[1]:10s} {d[2]:.2f} {d[3]:.8f} {d[4]: .8f} {d[5]:.8f} {d[6]:.8f} {d[7]:.8f}'.format(d=trade))
    df_trades.loc[len(df_trades)] = trade
    fn =  'trades-' + robot_name + '_' + str(rsiWin) + '_' + str(rsiBuy) + '_' + str(rsiSell) + '.csv'
    df_trades.to_csv(fn , index = False)
    if trade[1] == "Entry" :
        order_type = 'sell'
    else:
        order_type = 'buy'
        
        
def make_order(dp, order_type, market_name, numShares, price):
    if not DEBUG :
        if order_type == 'buy' :
            order = dp.trade_buy(market_name, 'LIMIT', numShares, price, TIMEINEFFECT_IMMEDIATE_OR_CANCEL)
        elif order_type == 'sell' :
            order = dp.trade_sell(market_name, 'LIMIT', numShares, price, TIMEINEFFECT_IMMEDIATE_OR_CANCEL)
        
        if response_fail(order):
            return  (False, 1,0,0,0)

        uuid =  order['result']['OrderId']          

        orders = dp.get_order_history(market_name)
        if response_fail(orders):
            return (False, 2,0,0,0)
        
        success = False
        counter = 0
        while (not success or counter <= 3 ):
            time.sleep(1)
            counter += 1 
            orders = dp.get_order_history(market_name)
            if response_fail(orders):
                return (False, 2,0,0,0)

            orders = json_normalize(orders['result'])
            filled_order = orders[orders['OrderUuid'] == uuid][['Quantity','PricePerUnit', 'Price', 'QuantityRemaining']]
            success = bool(filled_order.shape[0])
        
        if not success:
            return (False, 3,0,0,0)
        else:
            filled_order = filled_order.to_records(index = False)[0]    
    else:
        success = True
        filled_order = (numShares, numShares * price, price, 0)
        
    return (success, *filled_order)

In [25]:
data_path = read_data_path()
df_trades = pd.DataFrame(columns = ['Date', 'Trade type', 'NumShares', 'Price', 'Close', 'cumProfit', 'curCapital', 'Deposit'])

commision = 0.001

markets = pd.read_csv('markets_in_trade.csv')
market_position = np.zeros(markets.shape[0])
tf_since_entry = np.zeros(markets.shape[0])
cur_capital = np.zeros(markets.shape[0])
num_shares = np.zeros(markets.shape[0])
cur_capital += start_capital

cum_profit = 0
deposit = 0
last_time = ''

stop= False
hold = 30
fault_count = 0



In [26]:
cur_capital

array([0.01, 0.01, 0.01])

In [2]:
while not stop:
    time.sleep (exchange.rateLimit / 1000) # time.sleep wants seconds
    for ind in range(markets.shape[0]):
        market = markets.iloc[ind]['market']
        tf = markets.iloc[ind]['tf']
        df = pd.read_csv(data_path + '/Cash/' + market + '_' + tf + '.csv'  )

        data_len = df.shape[0]
        tf_closes = df["C"].as_matrix()
        
        signals = rsi_01(tf_closes, rsi_win, rsi_buy, rsi_sell)
        
        price = tf_closes[-1]
        #Long entry 
        if  market_position[ind] == 0 and signals[-1] = 1:

            num_shares[ind] = cur_capital[ind] / price * (1-commision)     

            flag, quantity, price, profit, quantityRemaining =  make_order(dp, 'buy', market_name, numShares, price)
            if flag :
                if quantityRemaining > 0:
                    print("Entry numShares: ", numShares, "Quantity: ", quantity, "Quantity Remaining:", quantityRemaining)

                buySum = profit + curCapital * commision
                curCapital = 0;

                market_position[ind] = 1
                tf_since_entry[ind] = 0    
                entryPrice = price  

                print("Entry")
                write_trades(df_trades, [myDate[-1], "Entry", numShares, price,  myClose[-1], cumProfit, curCapital, deposit])
            else:
                print("buy fail:", quantity)
        #Long Exit Profit 
        if market_position[ind] == 1 and tf_since_entry[ind] > 1 and signals[-1] == 2:
            price = get_price_orderbook('Exit')

            flag, quantity,price, profit, quantityRemaining = make_order(dp, 'sell', market_name, numShares, price)
            if flag:
                if quantityRemaining > 0:
                    print("Entry numShares: ", numShares, "Quantity: ", quantity, "Quantity Remaining:", quantityRemaining)


                pure_profit = profit - buySum
                cumProfit += pure_profit
                deposit_sum = max(0, pure_profit) * deposit_percent
                deposit += deposit_sum
                curCapital = profit - deposit_sum

                market_position[ind] = 0

                write_trades(df_trades, [myDate[-1], "Exit", numShares, price, myClose[-1] , cumProfit, curCapital, deposit])
            else:
                print("sell fail : ", quantity)
      #------------------Post processing---------------------------------------------------

        if last_time != df.iloc[-1]['T']:
            last_time = df.iloc[-1]['T']
            #print(str(datetime.now()))
            #print(df.iloc[-3:][['T','C']])

             tf_since_entry[ind] += 1           

        log_filename = 'Logs/log-' + robot_name + market.split('-')[1] + '.' + tf  +  '_' + str(rsi_win) + '_' + str(rsi_buy) + '_' + str(rsi_sell) + '.log'    
        with open(log_filename, 'w') as log_file:
            log_file.write("%s, MP: %s, RSI: %s" % (myDate[-1], market_position[ind], rsiVal))

        with open("stop.txt", 'r') as file:
            content = file.read()
            content = content.replace(' ', '')
            content = content.split('\n')
            stop_flag = [s for s in content if robot_name in s]
            stop_flag = stop_flag[0].split('=')[1]
            stop = bool(int(stop_flag))
        
        
    #Продаем если у нас есть альты
    if stop and marketPosition == 1:
        print(stop)
        price = get_price_orderbook('Stop Robot')
        flag, quantity,price, profit, quantityRemaining = make_order(dp, 'sell', market_name, numShares, price)
        if flag:
            if quantityRemaining > 0:
                print("Entry numShares: ", numShares, "Quantity: ", quantity, "Quantity Remaining:", quantityRemaining)
                
            cumProfit += profit - buySum
            curCapital = profit 
            marketPosition = 0

            write_trades(df_trades, [myDate[-1], "Stop Robot", numShares, price, myClose[-1], cumProfit, curCapital, deposit])
        else:
            print("sell fail: ", quantity)

NameError: name 'exchange' is not defined